In [ ]:
import json
import requests
from urllib.parse import urlencode, quote

In [ ]:
import requests
from urllib.parse import urlencode


class FortiAPIClient():

    def __init__(self, host, session=None, protocol='http', timeout=12):
        self._host = host
        self._session = session or requests.session()
        self._protocol = protocol
        self._timeout = timeout

    def __repr__(self):
        return f'{self.__class__.__name__}({self._host.__repr__()})'

    def __enter__(self):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        self.close()

    @property
    def host(self):
        return self._host

    @property
    def session(self):
        return self._session

    @property
    def protocol(self):
        return self._protocol

    @property
    def timeout(self):
        return self._timeout

    @property
    def url_root(self):
        url_root = f'{self._protocol}://{self._host}'
        return url_root

    def login(self, username, password, path='/logincheck', ajax=1):
        """
        ajax: Format the response for easier parsing. Enable using 1.
        """
        url = self.url_root + path
        data = {
            'username': username,
            'secretkey': password,
            'ajax': ajax,
        }
        encoded_data = urlencode(data)
        response = self._session.post(
            url=url,
            data=encoded_data,
            timeout=self._timeout
        )
        return response

    def get(self, path, params={}):
        url = self.url_root + path
        response = self._session.get(
            url=url,
            params=params,
            timeout=self.timeout,
        )
        return response

    def post(self, path, data={}, json=''):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.post(
            url=url,
            headers=headers,
            data=data,
            json=json,
            timeout=self.timeout,
        )
        return response

    def put(self, path, data={}, json=''):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.put(
            url=url,
            headers=headers,
            data=data,
            json=json,
            timeout=self.timeout,
        )
        return response

    def delete(self, path):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.delete(
            url=url,
            headers=headers,
            timeout=self.timeout,
        )
        return response

    def logout(self, path='/logout'):
        url = self.url_root + path
        response = self._session.post(
            url=url,
            timeout=self._timeout
        )
        return response

    def close(self):
        self.logout()
        self._session.close()

In [ ]:
# Usage

In [ ]:
CREDENTIALS = {
    'host': '150.117.123.248',
    'users': {
        'admin': {
            'username': 'api_admin',
            'password': ''
        }
    }
}

In [ ]:
# Login - Admin
client = FortiAPIClient('150.117.123.248')
r = client.login(
    username=CREDENTIALS['users']['admin']['username'],
    password=CREDENTIALS['users']['admin']['password']
)

In [ ]:
# Login check
status_code = r.text[:1]
descriptions = {
    '0': 'Log in failure. Most likely an incorrect username/password combo.',
    '1': 'Successful log in',
    '2': 'Admin is now locked out',
    '3': 'Two-factor Authentication is needed',
}
msg = descriptions.get(status_code, 'Unknown error. Can you log in manually?')
is_logged_in = (status_code == '1')
if not is_logged_in:
    raise ValueError(msg)

In [ ]:
# Get
response = client.get(
    path='/api/v2/cmdb/firewall/address',
    params={'format': 'name|subnet'}
)
print(response.text)

In [ ]:
# Get - with filter
response = client.get(
    path='/api/v2/cmdb/firewall/address',
    params={
        'format': 'name|comment',
        'filter': r'comment=@\"created_by\": \"j3y\"',
    }
)
print(response.text)

In [ ]:
# Read & write
data = response.json()
# Do something with data
# ....
# ....

In [ ]:
# Post
response = client.post(
    path='/api/v2/cmdb/firewall/address',
    json={
        'name': 'address 10.210.201.168/32',
        'type': 'ipmask',
        'subnet': '10.210.201.168 255.255.255.255',
    }
)
print(response.text)

In [ ]:
# Put
response = client.put(
    path='/api/v2/cmdb/firewall/address' + '/' + quote('address 10.210.201.168/32', safe=''),
    json={
        'name': 'address__10.210.201.168/32',
    }
)
print(response.json())

In [ ]:
# Delete
response = client.delete(
    path='/api/v2/cmdb/firewall/address' + '/' + quote('address__10.210.201.168/32', safe=''),
)
print(response.text)